# Data Preparation

This notebook loads and cleans raw NBA player, team, and All-Star selection data to prepare it for modeling. It resolves inconsistencies across data sources, removes or corrects problematic rows, and merges season-level statistics into a single, clean dataset. 

At the end, this notebook outputs:
- `X_data.pkl`: cleaned feature dataset for modeling
- `y_data.pkl`: binary target variable indicating All-Star selection
- `clean_data_reference.csv`: full combined dataset for manual review

In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
# define columns to use from our raw data
player_cols = [
    "season", "player_id", "player", "experience", "tm", "g", "mp", "x3p","x3pa",
    "x2p", "x2pa", "ft", "fta", "orb", "drb", "ast", "stl", "blk", "pts"
]
team_cols = ["season", "abbreviation", "w", "l"]
all_star_cols = ["player", "season"]

In [3]:
# load csv data
players = pd.read_csv("../data/raw/Player Totals.csv", usecols=player_cols)
teams = pd.read_csv("../data/raw/Team Summaries.csv", usecols=team_cols)
all_stars = pd.read_csv("../data/raw/All-Star Selections.csv", usecols=all_star_cols)

# filter out 2025 (data doesn't include 2025 all-star selections) and pre-merger era
players = players[(players["season"] >= 1977) & (players["season"] < 2025)]
teams = teams[(teams["season"] >= 1977) & (teams["season"] < 2025)]
all_stars = all_stars[(all_stars["season"] >= 1977) & (all_stars["season"] < 2025)]

In [4]:
players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24955 entries, 719 to 25673
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      24955 non-null  int64  
 1   player_id   24951 non-null  float64
 2   player      24955 non-null  object 
 3   experience  24955 non-null  float64
 4   tm          24955 non-null  object 
 5   g           24955 non-null  int64  
 6   mp          24955 non-null  float64
 7   x3p         23880 non-null  float64
 8   x3pa        23880 non-null  float64
 9   x2p         24955 non-null  int64  
 10  x2pa        24955 non-null  int64  
 11  ft          24955 non-null  int64  
 12  fta         24955 non-null  int64  
 13  orb         24955 non-null  float64
 14  drb         24955 non-null  float64
 15  ast         24955 non-null  int64  
 16  stl         24955 non-null  float64
 17  blk         24955 non-null  float64
 18  pts         24955 non-null  int64  
dtypes: float64(9), int64(8), obj

In [5]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1368 entries, 31 to 1398
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   season        1368 non-null   int64  
 1   abbreviation  1320 non-null   object 
 2   w             1320 non-null   float64
 3   l             1320 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.4+ KB


In [6]:
all_stars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1202 entries, 0 to 1201
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   player  1202 non-null   object
 1   season  1202 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 28.2+ KB


### Basic Cleaning and Typecasting

We begin by addressing missing values and ensuring appropriate data types for modeling. This includes dropping problematic rows, filling nulls, and converting numeric columns to integers.

In [7]:
# drop rows with select missing data
players = players.dropna(subset=["player_id"])
teams = teams.dropna()

# fill NaN with 0
players['x3p'] = players['x3p'].fillna(0)
players['x3pa'] = players['x3pa'].fillna(0)

# convert columns to int
player_int_cols = ['player_id', 'experience', 'g', 'x3p', 'x3pa', 'x2p', 'x2pa', 'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'pts']
players[player_int_cols] = players[player_int_cols].astype('int')
teams[['w', 'l']] = teams[['w', 'l']].astype('int')

# check for remaining nan values
players.isnull().sum().sum(), teams.isnull().sum().sum(), all_stars.isnull().sum().sum()

(np.int64(0), np.int64(0), np.int64(0))

### Consolidating Multi-Team Player-Seasons

Players who changed teams mid-season appear multiple times in the player dataset. These rows must be merged for accurate analysis:
- We assign the team with the most games played as the player's primary team.
- We keep only the total stat row for that player-season (labeled as `TOT`), retaining the one with the highest minutes played.

In [8]:
# create a dictionary of teams and games played by player-season
player_team_data = defaultdict(lambda: defaultdict(list))
for _, row in players.iterrows():
    if row["tm"] == "TOT":
        continue  # skip totaled rows
    season = row["season"]
    player_id = row["player_id"]
    tm = row["tm"]
    g = row["g"]

    player_team_data[season][player_id].append({"tm": tm, "g": g})

# set primary team in any player-season aggregated rows (total rows for players who had multiple teams in a season)
for idx, row in players.iterrows():
    if row["tm"] == "TOT":
        primary_tm = max(player_team_data[row["season"]][row["player_id"]], key=lambda x: x["g"])["tm"]
        players.loc[idx, "tm"] = primary_tm

# keep only the total row for player-seasons with multiple entries (keeping row with the highest minutes played)
players = players.loc[players.groupby(["player_id", "season"])["mp"].idxmax()].reset_index(drop=True)

### Resolving Name Conflicts in All-Star Data

Since the All-Star selections file does not include `player_id`, players with the same name in the same season could be misclassified. We check for these name collisions and confirm whether any conflicting names could lead to incorrect All-Star labeling.

In [9]:
# check for players of the same name playing in the same season
name_id_check = players.groupby(["season", "player"])["player_id"].nunique()
conflicts = name_id_check[name_id_check > 1]

# check if conflicting player names are included in all-star list
conflict_names = conflicts.index.get_level_values("player").unique()
conflict_all_stars = all_stars[all_stars["player"].isin(conflict_names)]

print(conflicts, '\n\n', conflict_all_stars)

season  player         
1979    George Johnson     2
1980    George Johnson     2
1981    George Johnson     2
1982    Eddie Johnson      2
        George Johnson     2
1983    Eddie Johnson      2
        George Johnson     2
1984    Eddie Johnson      2
1985    Charles Jones      2
        Eddie Johnson      2
        George Johnson     2
1986    Charles Jones      2
        Eddie Johnson      2
        George Johnson     2
1987    Eddie Johnson      2
1988    Charles Jones      2
1989    Charles Jones      2
1990    Charles Smith      2
1991    Charles Smith      2
1995    Michael Smith      2
1996    Charles Smith      2
2008    Marcus Williams    2
2009    Marcus Williams    2
2013    Chris Johnson      2
2014    Tony Mitchell      2
Name: player_id, dtype: int64 

              player  season
1086  Eddie Johnson    1981
1113  Eddie Johnson    1980


Fortunately, the only conflict involves two players named Eddie Johnson. The Eddie Johnson named to the All-Star team did so before the other Eddie Johnson entered the league, so no correction is needed.

### Checking for Missing Player-Seasons

We now check whether there are any All-Star selections for player-seasons that do not exist in our player stats dataset.

In [10]:
# create sets of player-season pairs and print the all_star -> player difference
player_keys = set(zip(players['season'], players['player']))
all_star_keys = set(zip(all_stars['season'], all_stars['player']))
missing = all_star_keys - player_keys

print(missing)

{(1992, 'Magic Johnson')}


### Special Case: Magic Johnson in 1992

In 1992, Magic Johnson was selected to the All-Star Game despite retiring prior to the season due to his HIV diagnosis. While this was an iconic moment in NBA history, it does not reflect typical season performance and would distort our model. We'll remove this record from the All-Star selections.

In [11]:
# remove Magic's 1992 All-Star appearance
all_stars = all_stars[~((all_stars['season'] == 1992) & (all_stars['player'] == 'Magic Johnson'))]

### Merging Data Sources

With all datasets cleaned and aligned, we now merge player and team data to create a single modeling-ready dataset. We also generate the binary `all_star` target column based on the cleaned All-Star selections.

In [12]:
# create a clean, merged dataframe by adding team wins to each player-season
clean_data = players.merge(
    teams[['season', 'abbreviation', 'w']],
    left_on=['season', 'tm'],
    right_on=['season', 'abbreviation'],
    how='left'
).drop(columns='abbreviation')

In [13]:
# add all_star column and populate (1 if player-season resulted in All-Star selection else 0)
all_star_set = set(zip(all_stars['season'], all_stars['player']))
clean_data['all_star'] = clean_data.apply(
    lambda row: 1 if (row['season'], row['player']) in all_star_set else 0,
    axis=1
)

### Final Modeling Decisions

Before exporting the final data, we make a few decisions about which features to keep and how to represent them:
- **Season totals** are used instead of per-game stats, as they better reflect a player's full-season impact.
- **Shooting percentages** are excluded since they can be calculated from other columns and would be redundant.
- The **`tm`** column (team) is retained and one-hot encoded, as market and media presence may influence fan and coach voting.
- The **`season`** column is kept to provide historical context, acknowledging that All-Star criteria may shift over time.

In [14]:
# drop irrelevant columns for modeling and encode team
X = clean_data.drop(columns=['player', 'player_id', 'all_star'])
X = pd.get_dummies(X, columns=['tm'], drop_first=True)

# set target
y = clean_data['all_star']

Now we just need to save our data so we'll have nice, clean data for use in our modeling notebook.

In [16]:
# save dataframes as pickle in the clean folder
X.to_pickle('../data/clean/X_data.pkl')
y.to_pickle('../data/clean/y_data.pkl')

# also keeping clean_data as a csv for reference in case I want to inspect the data manually in excel (for instance)
clean_data.to_csv('../data/clean/clean_data_reference.csv', index=False)